# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_path = "Weather_Data.csv"
vacation_df = pd.read_csv(vacation_path)
vacation_df.dropna(inplace = True) 
vacation_df.head()



,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mataura,-46.1927,168.8643,11.93,98,10,0.38,NZ,1651046366
1,khatanga,71.9667,102.5000,-4.27,80,100,2.50,RU,1651046412
2,cape town,-33.9258,18.4232,19.39,63,0,2.06,ZA,1651046402
3,ushuaia,-54.8000,-68.3000,2.81,81,75,9.77,AR,1651046132
4,roald,62.5833,6.1333,5.74,56,75,1.03,NO,1651046468


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = vacation_df[["Latitude", "Longitude"]]
humidity = vacation_df['Humidity'].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,  weights=humidity, dissipating=False, max_intensity=max(vacation_df['Humidity']),point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_clean_data = vacation_df.loc[(vacation_df['Max Temp']<27) & (vacation_df['Max Temp']>21)]
vacation_clean_data = vacation_clean_data.loc[vacation_clean_data['Wind Speed']<15]
vacation_clean_data = vacation_clean_data.loc[vacation_clean_data['Cloudiness']==0]
vacation_clean_data


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,marsa matruh,31.3525,27.2453,21.77,44,0,3.32,EG,1651046646
90,guaymas,27.9333,-110.9000,21.29,51,0,2.43,MX,1651046667
92,faya,18.3851,42.4509,23.29,24,0,3.61,SA,1651046668
125,cabo san lucas,22.8909,-109.9124,23.06,38,0,2.68,MX,1651046138
239,las vegas,36.1750,-115.1372,24.68,13,0,3.60,US,1651046759
295,makakilo city,21.3469,-158.0858,23.18,71,0,6.17,US,1651046791
335,lavumisa,-27.3100,31.8920,21.32,50,0,3.24,SZ,1651046816
336,seoul,37.5683,126.9778,24.66,14,0,4.05,KR,1651046656
382,terrasini,38.1460,13.0844,24.15,60,0,1.03,IT,1651046847
407,corinth,37.9407,22.9573,21.05,60,0,1.74,GR,1651046585


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_clean_data
hotel_df['Hotel Name'] = ''
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
58,marsa matruh,31.3525,27.2453,21.77,44,0,3.32,EG,1651046646,
90,guaymas,27.9333,-110.9000,21.29,51,0,2.43,MX,1651046667,
92,faya,18.3851,42.4509,23.29,24,0,3.61,SA,1651046668,
125,cabo san lucas,22.8909,-109.9124,23.06,38,0,2.68,MX,1651046138,
239,las vegas,36.1750,-115.1372,24.68,13,0,3.60,US,1651046759,
295,makakilo city,21.3469,-158.0858,23.18,71,0,6.17,US,1651046791,
335,lavumisa,-27.3100,31.8920,21.32,50,0,3.24,SZ,1651046816,
336,seoul,37.5683,126.9778,24.66,14,0,4.05,KR,1651046656,
382,terrasini,38.1460,13.0844,24.15,60,0,1.03,IT,1651046847,
407,corinth,37.9407,22.9573,21.05,60,0,1.74,GR,1651046585,


In [7]:
params = {'key':g_key, 'radius' :5000, "types": "lodging"}


for index, row in hotel_df.iterrows():
    
    latitude= row['Latitude']
    longitude= row['Longitude']
    params['location'] = f'{latitude},{longitude}'

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_lat_lng = requests.get(url,params=params)
    hotel_lat_lng = hotel_lat_lng.json()
    results = hotel_lat_lng['results']
    try:
        print(f"Hotel close to you: {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Hotel close to you: Beau Site.
Hotel close to you: Holiday Inn Express Guaymas, an IHG Hotel.
Hotel close to you: املاك خالد الشعشوع.
Hotel close to you: Hotel Tesoro Los Cabos.
Hotel close to you: Plaza Hotel & Casino.
Hotel close to you: Marriott's Ko Olina Beach Club.
Hotel close to you: ROADSIDE B&B.
Hotel close to you: 롯데호텔 서울.
Hotel close to you: Florio Park Hotel.
Hotel close to you: Ephira Hotel.
Missing field/result... skipping.
Hotel close to you: Best Western Hotel Plaza Matamoros.


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
locations


,Latitude,Longitude
58,31.3525,27.2453
90,27.9333,-110.9000
92,18.3851,42.4509
125,22.8909,-109.9124
239,36.1750,-115.1372
295,21.3469,-158.0858
335,-27.3100,31.8920
336,37.5683,126.9778
382,38.1460,13.0844
407,37.9407,22.9573


In [10]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))